In [29]:
import pandas as pd

In [34]:
# Read in the Asx Sectors Excel file as a dictionary where each key is a sheet and each value is a dataframe of the sheet data
companies = pd.read_excel('data/ASX Sectors with Companies.xlsx', sheet_name = 'Sectors')
companies['ticker'] = companies.loc[:,'code'].str.replace('ASX:','')
# companies['ticker'] = companies['code'].str.replace('ASX:','')
companies['yahoo_ticker'] = companies['ticker'] + '.AX'
companies['url'] = 'https://www.listcorp.com/' + companies['market'] + '/' + companies['ticker']
print(companies.info())
print(companies.head())
# print(companies.describe())
# print(companies.columns)
# Get a series of the sectors
sector = companies['gics_sector_name']
# print(sector)
# get the number of each sector
# print(sector.value_counts())
print(companies['gics_industry_group_name'].value_counts())
# Get the unique industries for a specific sector
# print(companies[companies['sector']=='Real Estate']['industry'].unique())
# Get all the industries as a series
industry = companies['gics_industry_name']
# print(type(industry))
# print(industry.head(30))
unique_industries = industry.value_counts()
# # Returns a ndarray
# print(unique_industries.head(30))
# print(type(unique_industries))
# print(unique_industries['Metals & Mining'])
# Get the number of each industry
# sub_industry = companies['sub_industry']
# unique_sub_industry = sub_industry.value_counts()
# print(unique_sub_industry['Copper'])
# companies[['sector', 'ticker']].groupby(['sector']).count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   market                    1997 non-null   object
 1   gics_sector_name          1997 non-null   object
 2   sector_ticker             1997 non-null   object
 3   sector_ticker_yahoo       1997 non-null   object
 4   gics_industry_group_name  1997 non-null   object
 5   gics_industry_name        1997 non-null   object
 6   gics_sub_industry_name    1997 non-null   object
 7   code                      1980 non-null   object
 8   name                      1980 non-null   object
 9   ticker                    1980 non-null   object
 10  yahoo_ticker              1980 non-null   object
 11  url                       1980 non-null   object
dtypes: object(12)
memory usage: 187.3+ KB
None
  market gics_sector_name sector_ticker sector_ticker_yahoo  \
0    ASX           Energy 

In [41]:
# create a list of columns from the dataframe
table_columns = list(companies.columns)
print(table_columns)
columns = ",".join(table_columns)

print(columns)
# create VALUES('%s', '%s',...) one '%s' per column
# values = f"VALUES({','.join(['%s' for _ in ticker_list])})"
# print(values)
# what to do on conflict
conflict_columns = ", ".join(['EXCLUDED.' + column for column in table_columns])
print(conflict_columns)

['market', 'gics_sector_name', 'sector_ticker', 'sector_ticker_yahoo', 'gics_industry_group_name', 'gics_industry_name', 'gics_sub_industry_name', 'code', 'name', 'ticker', 'yahoo_ticker', 'url']
market,gics_sector_name,sector_ticker,sector_ticker_yahoo,gics_industry_group_name,gics_industry_name,gics_sub_industry_name,code,name,ticker,yahoo_ticker,url
EXCLUDED.market, EXCLUDED.gics_sector_name, EXCLUDED.sector_ticker, EXCLUDED.sector_ticker_yahoo, EXCLUDED.gics_industry_group_name, EXCLUDED.gics_industry_name, EXCLUDED.gics_sub_industry_name, EXCLUDED.code, EXCLUDED.name, EXCLUDED.ticker, EXCLUDED.yahoo_ticker, EXCLUDED.url


In [7]:
for row in companies.itertuples(name='none'):
    sector = row.sector
    industry_group = row.industry_group
    industry = row.industry
    sub_industry = row.sub_industry
    break

In [4]:
import yfinance as yf
import time
from datetime import datetime as dt
from datetime import timezone as tz
from dateutil.relativedelta import relativedelta
YEARS = 1
WAIT_TIME_IN_SECONDS = 1.0
now = dt.now(tz.utc)
# print(now)
end = f"{now.year}-{now.month}-{now.day}"
# print(end)
start_date = now - relativedelta(years=YEARS)
start = f"{start_date.year}-{start_date.month}-{start_date.day}"
# print(start)
sector_tickers = companies['sector_ticker_yahoo'].unique()
# print(sector_tickers)
# print(type(sector_tickers))
list_sector_tickers = list(sector_tickers)
# print(type(list_sector_tickers))
data_list = []
# print(list_sector_tickers)
for ticker in list_sector_tickers:
    # print(f'Ticker: {ticker}"')
    obj = yf.Ticker(ticker)
    ohlc = obj.history(start=start, end=end)
    ohlc['Ticker'] = ticker
    data_list.append(ohlc)
    time.sleep(WAIT_TIME_IN_SECONDS)
data = pd.concat(data_list)
# print(data.shape)
# print(data['Ticker'].value_counts())
# print(data.columns)
# print(data.head(3))
data = data.reset_index()
# print(data.columns)
data = data.set_index(['Ticker', 'Date'])
# print(data.head(3))
print(data.loc[['^AXEJ'],['Close']])

                                         Close
Ticker Date                                   
^AXEJ  2023-06-29 00:00:00+10:00  10787.700195
       2023-06-30 00:00:00+10:00  10827.799805
       2023-07-03 00:00:00+10:00  10917.799805
       2023-07-04 00:00:00+10:00  11044.200195
       2023-07-05 00:00:00+10:00  10983.900391
...                                        ...
       2024-06-24 00:00:00+10:00   9754.000000
       2024-06-25 00:00:00+10:00   9971.799805
       2024-06-26 00:00:00+10:00  10024.900391
       2024-06-27 00:00:00+10:00  10031.099609
       2024-06-28 00:00:00+10:00  10031.500000

[254 rows x 1 columns]


In [5]:
import yfinance as yf
import time
from datetime import datetime as dt
from datetime import timezone as tz
from dateutil.relativedelta import relativedelta
YEARS = 1
WAIT_TIME_IN_SECONDS = 1.0
now = dt.now(tz.utc)
# print(now)
end = f"{now.year}-{now.month}-{now.day}"
# print(end)
start_date = now - relativedelta(years=YEARS)
start = f"{start_date.year}-{start_date.month}-{start_date.day}"

# Define a list of ticker symbols to download
tickerStrings = ['AAPL', 'MSFT']

# Download 2 days of data for each ticker, grouping by 'Ticker' to structure the DataFrame with multi-level columns
df = yf.download(tickerStrings, start=start, end=end, group_by='Ticker', period='1d')
# print(df.info())
# print(df.head())
# Transform the DataFrame: stack the ticker symbols to create a multi-index (Date, Ticker), then reset the 'Ticker' level to turn it into a column
# df = df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
df = df.stack(level=0, future_stack=True).rename_axis(['Date', 'Ticker']).sort_index(level=['Ticker', 'Date'])
print('\n')
print(df.head())

[*********************100%%**********************]  2 of 2 completed



Price                    Open        High         Low       Close   Adj Close  \
Date       Ticker                                                               
2023-06-29 AAPL    189.080002  190.070007  188.940002  189.589996  188.588440   
2023-06-30 AAPL    191.630005  194.479996  191.259995  193.970001  192.945328   
2023-07-03 AAPL    193.779999  193.880005  191.759995  192.460007  191.443298   
2023-07-05 AAPL    191.570007  192.979996  190.619995  191.330002  190.319260   
2023-07-06 AAPL    189.839996  192.020004  189.199997  191.809998  190.796722   

Price                Volume  
Date       Ticker            
2023-06-29 AAPL    46347300  
2023-06-30 AAPL    85069600  
2023-07-03 AAPL    31458200  
2023-07-05 AAPL    46920300  
2023-07-06 AAPL    45094300  


In [10]:
df = yf.download(tickerStrings, start=start, end=end, group_by='ticker', period='1d')
print(df.info())
print(df.head())

[*********************100%%**********************]  2 of 2 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252 entries, 2023-06-29 to 2024-06-28
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (AAPL, Open)       252 non-null    float64
 1   (AAPL, High)       252 non-null    float64
 2   (AAPL, Low)        252 non-null    float64
 3   (AAPL, Close)      252 non-null    float64
 4   (AAPL, Adj Close)  252 non-null    float64
 5   (AAPL, Volume)     252 non-null    int64  
 6   (MSFT, Open)       252 non-null    float64
 7   (MSFT, High)       252 non-null    float64
 8   (MSFT, Low)        252 non-null    float64
 9   (MSFT, Close)      252 non-null    float64
 10  (MSFT, Adj Close)  252 non-null    float64
 11  (MSFT, Volume)     252 non-null    int64  
dtypes: float64(10), int64(2)
memory usage: 25.6 KB
None
Ticker            AAPL                                                  \
Price             Open        High         Low       Close   Adj Close   

In [11]:
df = yf.download(tickerStrings, start=start, end=end, period='1d')
print(df.info())
print(df.head())

[*********************100%%**********************]  2 of 2 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252 entries, 2023-06-29 to 2024-06-28
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Adj Close, AAPL)  252 non-null    float64
 1   (Adj Close, MSFT)  252 non-null    float64
 2   (Close, AAPL)      252 non-null    float64
 3   (Close, MSFT)      252 non-null    float64
 4   (High, AAPL)       252 non-null    float64
 5   (High, MSFT)       252 non-null    float64
 6   (Low, AAPL)        252 non-null    float64
 7   (Low, MSFT)        252 non-null    float64
 8   (Open, AAPL)       252 non-null    float64
 9   (Open, MSFT)       252 non-null    float64
 10  (Volume, AAPL)     252 non-null    int64  
 11  (Volume, MSFT)     252 non-null    int64  
dtypes: float64(10), int64(2)
memory usage: 25.6 KB
None
Price        Adj Close                   Close                    High  \
Ticker            AAPL        MSFT        AAPL        MSFT        AAPL   